In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('__file__')) + '/../../../')
from acaisdk.file import File
from acaisdk.project import Project
from acaisdk.fileset import FileSet
from acaisdk.job import Job
from acaisdk.meta import *
from acaisdk.utils import utils
from acaisdk import credentials

utils.DEBUG = True  # print debug messages
workspace = os.path.dirname(os.path.realpath('__file__'))  # get current directory

In [19]:
# Make your changes here
project_id = "automl_example"
root_token = 'EmDlCTBF1ppONSciYVd03M9xkmF6hFqW'
project_admin = 'joannc'
user = 'joannc'

In [20]:
# Create project and user
r = Project.create_project(project_id, root_token, project_admin)
# Login is done automatically upon user creation
r = Project.create_user(project_id, r['project_admin_token'], user)  

Running request: 35.237.138.197 80 credential create_project
POST data {"project_id": "automl_example", "admin_token": "EmDlCTBF1ppONSciYVd03M9xkmF6hFqW", "project_admin_name": "joannc"}
Running request: 35.237.138.197 80 credential create_user
POST data {"project_id": "automl_example", "admin_token": "D5cSf8bgUYVfGdyaclG5Gl0y17ZGnniN", "user_name": "joannc"}
Logged in with token NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM


In [30]:
# you can inspect your token in multiple ways
print(r['user_token'])
print(os.environ['ACAI_TOKEN'])
print(credentials.get_credentials())

NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM
NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM
{'token': 'NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM'}


In [35]:
credentials.login('NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM')

In [75]:
# Upload code
code = os.path.join(workspace, 'Archive.zip')
File.upload({code: '/Archive.zip'})

Running request: 35.237.138.197 80 storage start_file_upload_session
POST data {"paths": ["/Archive.zip"], "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}
[('/Users/Joann/Desktop/CMU/Capstone/code/acai-systems/acaisdk/acaisdk/example/example1/Archive.zip', '/Archive.zip')]
r =  <Response [200]>
Uploaded /Users/Joann/Desktop/CMU/Capstone/code/acai-systems/acaisdk/acaisdk/example/example1/Archive.zip to /Archive.zip
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 2535, "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 2535, "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}
Running request: 35.237.138.197 80 storage poll_file_upload_session
GET query {"session_id": 2535, "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}
Running request: 35.237.138.197 80 storage finish_file_upload_session
POST data {"session_id": 2535, "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}


[('/Users/Joann/Desktop/CMU/Capstone/code/acai-systems/acaisdk/acaisdk/example/example1/Archive.zip',
  'Archive.zip:9')]

In [66]:
# Upload input files and create file set
input_dir = os.path.join(workspace, 'SquadData')
File.convert_to_file_mapping([input_dir], 'SquadData2/') \
    .files_to_upload \
    .upload() \
    .as_new_file_set('squad3.works')

Running request: 35.237.138.197 80 storage start_file_upload_session
POST data {"paths": ["SquadData2/requirements.txt", "SquadData2/dev.json", "SquadData2/train.json"], "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}
[('/Users/Joann/Desktop/CMU/Capstone/code/acai-systems/acaisdk/acaisdk/example/example1/SquadData/requirements.txt', 'SquadData2/requirements.txt'), ('/Users/Joann/Desktop/CMU/Capstone/code/acai-systems/acaisdk/acaisdk/example/example1/SquadData/dev.json', 'SquadData2/dev.json'), ('/Users/Joann/Desktop/CMU/Capstone/code/acai-systems/acaisdk/acaisdk/example/example1/SquadData/train.json', 'SquadData2/train.json')]
r =  <Response [200]>
Uploaded /Users/Joann/Desktop/CMU/Capstone/code/acai-systems/acaisdk/acaisdk/example/example1/SquadData/requirements.txt to SquadData2/requirements.txt
r =  <Response [200]>
Uploaded /Users/Joann/Desktop/CMU/Capstone/code/acai-systems/acaisdk/acaisdk/example/example1/SquadData/dev.json to SquadData2/dev.json
r =  <Response [200]>
Uploaded /User

{'id': 'squad3.works:1',
 'files': ['SquadData2/train.json:1',
  'SquadData2/dev.json:1',
  'SquadData2/requirements.txt:1']}

In [76]:
# Run a job
job_setting = {
    "v_cpu": "0.2",
    "memory": "1024Mi",
    "gpu": "0",
    "command": "mkdir -p ./output/ &&(pip install -r SquadData2/requirements.txt) && (python3 local_pipeline.py ./SquadData2/ ./output/)",
    "container_image": "pytorch/pytorch",
    'input_file_set': 'squad3.works',
    'output_path': './output/',
    'code': '/Archive.zip',
    'description': 'do bm 25',
    'name': 'bm25_job'
}

j = Job().with_attributes(job_setting).run()

Running request: 35.237.138.197 80 storage resolve_file_set
GET query {"vague_name": "squad3.works", "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}
Running request: 35.237.138.197 80 job_registry new_job
POST data {"name": "bm25_job", "input_file_set": "squad3.works:1", "output_path": "./output/", "code": "/Archive.zip", "command": "mkdir -p ./output/ &&(pip install -r SquadData2/requirements.txt) && (python3 local_pipeline.py ./SquadData2/ ./output/)", "container_image": "pytorch/pytorch", "description": "do bm 25", "v_cpu": "0.2", "gpu": "0", "memory": "1024Mi", "job_status": null, "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}
{'status': {'message': 'launching'}, 'job': {'name': 'bm25_job', 'code': '/Archive.zip', 'command': 'mkdir -p ./output/ &&(pip install -r SquadData2/requirements.txt) && (python3 local_pipeline.py ./SquadData2/ ./output/)', 'description': 'do bm 25', 'gpu': '0', 'memory': '1024Mi', 'hidden': False, 'type': 'DEFAULT', 'vcpu': '0.2', 'id': 191, 'input_file_set': 'sq

In [27]:
# Take a look at what's in the output folder
File.list_dir('/my_output')

Running request: 35.237.138.197 80 storage list_directory
GET query {"directory_path": "/my_output", "token": "NfnsQzfk70WQ2u9qZMtqJdh6o8VRzuEM"}


RemoteException: b'edu.cmu.cs.mcds.acai.storage.controller.RequestFailedException: Entity my_output:0 does not exist.\n\tat edu.cmu.cs.mcds.acai.storage.model.FileRepository.resolveEntity(FileRepository.java:136)\n\tat java.lang.invoke.MethodHandle.invokeWithArguments(MethodHandle.java:627)\n\tat org.springframework.data.projection.DefaultMethodInvokingMethodInterceptor.invoke(DefaultMethodInvokingMethodInterceptor.java:65)\n\tat org.springframework.aop.framework.ReflectiveMethodInvocation.proceed(ReflectiveMethodInvocation.java:186)\n\tat org.springframework.transaction.interceptor.TransactionAspectSupport.invokeWithinTransaction(TransactionAspectSupport.java:294)\n\tat org.springframework.transaction.interceptor.TransactionInterceptor.invoke(TransactionInterceptor.java:98)\n\tat org.springframework.aop.framework.ReflectiveMethodInvocation.proceed(ReflectiveMethodInvocation.java:186)\n\tat org.springframework.dao.support.PersistenceExceptionTranslationInterceptor.invoke(PersistenceExc

In [ ]:
# Download the result to local device
File.download({'/my_output/wordcount.txt': workspace})

In [ ]:
# Inspect the 50 most frequent words
with open(os.path.join(workspace, 'wordcount.txt')) as f:
    for i, l in enumerate(f):
        if i >= 50:
            break
        print(l, end='')